In [1]:
import numpy as np
import pandas as pd

In [3]:
df_t = pd.read_csv('../final/joinedClimateJan.csv')
df_avg_electricity = pd.read_csv('../avg_electricity/avg_electricity_jan.csv')

In [7]:
df = pd.merge(df_t, df_avg_electricity, left_on=['TOWNNAME', 'VILLNAME'], right_on=['鄉鎮名稱', '村里名稱'],how='left')


In [12]:
result_x = pd.DataFrame()
result_y = pd.DataFrame()

for i in range(11,15):
    year = i
    col_list = []
    rename_dict = {f"人均用電量_20{year}":"人均用電量"}
    x_col = []

    for a in range(24):
        col_list.append(f'T_{year}_{a}')
        x_col.append(f'T_{a}')
        rename_dict[f'T_{year}_{a}'] = f'T_{a}'

    df_temp = df[[f"人均用電量_20{year}"] + col_list]
    df_temp=df_temp.dropna(axis=0)
    df_temp = df_temp.rename(columns=rename_dict)

    df_x_temp = df_temp[x_col]

    df_y_temp = df_temp[["人均用電量"]]
    result_x = pd.concat([result_x, df_x_temp],axis=0)
    result_y = pd.concat([result_y, df_y_temp],axis=0)

In [35]:
taipei_center = ['松山區','信義區','大安區','中山區','中正區']
x = result_x
y = result_y["人均用電量"]

In [36]:
arr_X = x.to_numpy()
arr_y = y.to_numpy()

In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [37]:
lm = LinearRegression()
lm.fit(arr_X, arr_y)

# 印出係數
print(lm.coef_)

# 印出截距
print(lm.intercept_ )

[ -92.73167589   42.2154081  -126.13926283  -13.05033302  119.61680049
  -98.50734164  134.00076765 -121.77185398  -29.74836473   37.21545672
   26.02760591    5.8597355    48.7659078   -24.27105294   48.25052859
  -63.77545183 -103.69868675  -92.06701804   87.4328067   131.05763607
 -150.40751343  -35.77304293  108.43313607   99.79684129]
463.3435641702835


In [38]:
# 模型績效
mse = np.mean((lm.predict(arr_X) - arr_y) ** 2)
r_squared = lm.score(arr_X, arr_y)
adj_r_squared = r_squared - (1 - r_squared) * (arr_X.shape[1] / (arr_X.shape[0] - arr_X.shape[1] - 1))

# 印出模型績效
print(mse)
print(r_squared)
print(adj_r_squared)

14420.89973122507
0.05714237417622037
0.02280461542087786


In [24]:
from sklearn.feature_selection import f_regression

In [41]:
# 印出 p-value
print(f_regression(arr_X, arr_y)[1])

[0.3336261  0.30313547 0.31855324 0.40037706 0.46733618 0.56096165
 0.64018973 0.55675298 0.17538803 0.28298436 0.26657727 0.29328681
 0.30079141 0.28279694 0.30200246 0.29738338 0.26347837 0.26695067
 0.41852597 0.41483084 0.43775168 0.43771756 0.43399017 0.39161029]


In [45]:
import statsmodels.api as sm


In [46]:
#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

d:\Users\63742\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [49]:
#calculate sse
sse = np.sum((model.fittedvalues - y)**2)
print(sse)

#calculate ssr
ssr = np.sum((model.fittedvalues - y.mean())**2)
print(ssr)

#calculate sst
sst = ssr + sse
print(sst)

9863895.416157946
597806.4845291204
10461701.900687067


In [51]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  人均用電量   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.664
Date:                Wed, 13 Jul 2022   Prob (F-statistic):             0.0247
Time:                        17:38:54   Log-Likelihood:                -4245.7
No. Observations:                 684   AIC:                             8541.
Df Residuals:                     659   BIC:                             8655.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        463.3436    123.317      3.757      0.000     221.202     705.485
T_0          -92.7317     86.171     -1.076      0.282    -261.935      76.472
T_1           42.2154     85.581      0.493      0.622    -125.829     210.260
T_2         -126.1393     80.082     -1.575      0.116    -283.386      31.108
T_3          -13.0503     46.468     -0.281      0.779    -104.294      78.193
T_4          119.6168     82.375      1.452      0.147     -42.132     281.366
T_5          -98.5073     79.174     -1.244      0.214    -253.970      56.956
T_6          134.0008     72.236      1.855      0.064      -7.840     275.841
T_7         -121.7719     56.326     -2.162      0.031    -232.372     -11.172
T_8          -29.7484     33.299     -0.893      0.372     -95.133      35.636
T_9           37.2155     43.402      0.857      0.391     -48.007     122.438
T_10          26.0276     37.123      0.701      0.483     -46.867      98.922
T_11           5.8597     40.321      0.145      0.884     -73.313      85.033
T_12          48.7659     38.339      1.272      0.204     -26.516     124.048
T_13         -24.2711     40.169     -0.604      0.546    -103.147      54.604
T_14          48.2505     37.534      1.286      0.199     -25.451     121.952
T_15         -63.7755     39.397     -1.619      0.106    -141.135      13.584
T_16        -103.6987     39.778     -2.607      0.009    -181.806     -25.591
T_17         -92.0670     56.620     -1.626      0.104    -203.245      19.111
T_18          87.4328     55.813      1.567      0.118     -22.161     197.027
T_19         131.0576    107.723      1.217      0.224     -80.465     342.580
T_20        -150.4075     94.137     -1.598      0.111    -335.253      34.438
T_21         -35.7730     85.572     -0.418      0.676    -203.799     132.253
T_22         108.4331     72.796      1.490      0.137     -34.507     251.373
T_23          99.7968     71.779      1.390      0.165     -41.145     240.739
==============================================================================
Omnibus:                       47.124   Durbin-Watson:                   1.850
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.672
Skew:                           0.253   Prob(JB):                     1.19e-05
Kurtosis:                       2.266   Cond. No.                     2.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""